In [1]:
# import pandas as pd
# train_df = pd.read_json("/kaggle/input/my-redial/train_data.jsonl", lines=True)
# test_df = pd.read_json("/kaggle/input/my-redial/test_data.jsonl", lines=True)

In [2]:
# pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

/home/pc/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/pc/miniforge3/envs/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import torch
movie_db = torch.load('/home/pc/Documents/CRS/data/full_redial/movie_db_20_50')
len(movie_db)

6924

In [5]:
train_data = torch.load('/home/pc/Documents/CRS/data/full_redial/train_data')
# test_data = torch.load('/kaggle/input/efficient-processed/efficient_processed/test_data_processed')

In [6]:
def add_like(data):
    new_data = []
    for conv_id, conv in data:
        new_conv = []
        for turn, movie_lst in conv:
            if movie_lst:
                new_conv.append((turn, [(movie_id, 1) for movie_id in movie_lst]))
            else:
                new_conv.append((turn, None))
        new_data.append((conv_id, new_conv))
    return new_data
train_data = add_like(train_data)
# train_data[0]

In [7]:
def replace_movie_ids(turn_string, movies_lst, movie_db):
    # Split the turn string into parts to isolate [MOVIE_ID] placeholders
    parts = turn_string.split("[MOVIE_ID]")
    new_turn_string = parts[0]

    # Iterate over each part and the corresponding movie ID
    for part, (movie_id, liked) in zip(parts[1:], movies_lst):
        # Get the movie title from movie_db
        movie_title = movie_db[movie_id].split("[SEP]")[0].strip()
        # Append the movie title and the next part of the turn string
        new_turn_string += movie_title + part

    return new_turn_string
    
def add_movie(data, movie_db):
    new_data = []
    for conv in data:
        new_conv = []
        for turn in conv[1]:
            turn_string = turn[0]
            movies_lst = turn[1]
            if movies_lst is not None:
                turn_string = replace_movie_ids(turn_string, movies_lst, movie_db)
            new_conv.append((turn_string, movies_lst))
        new_data.append((conv[0], new_conv))
    return new_data


new_train_data = add_movie(train_data, movie_db)
# new_test_data = add_movie(test_data, movie_db)

In [8]:
import pandas as pd
def create_df(lst):
    df = pd.DataFrame()
    conv_ids = []
    user_ids = []
    convs = []
    for conv_id, (user_id, conv) in enumerate(lst):
        conv_ids.append(conv_id)
        user_ids.append(user_id)
        convs.append(conv)
    df['conv_id'] = conv_ids
    df['user_id'] = user_ids
    df['conv'] = convs
    return df

train_df = create_df(new_train_data)
# test_df = create_df(new_test_data)

In [9]:
def extract_movie_ids(conversation):
    movie_ids = []
    for message, movies in conversation:
        if movies:  # Check if the movies list is not None
            movie_ids.extend([movie_id for movie_id, _ in movies])  # Extract only the movie IDs
    return list(set(movie_ids))  # Return unique movie IDs

def get_plain_text(conversation):
    plain_text = ''
    for message, movies in conversation:
        plain_text = plain_text + message + ' /n '
    return plain_text
# Create a new column for all movie IDs
train_df["all_movie_id"] = train_df["conv"].apply(extract_movie_ids)
train_df["message"] = train_df["conv"].apply(get_plain_text)

# test_df["all_movie_id"] = test_df["conv"].apply(extract_movie_ids)
# test_df["message"] = test_df["conv"].apply(get_plain_text)

# Display the result
train_df

,conv_id,user_id,conv,all_movie_id,message
0,0,0,"[(B: Hi there, how are you? I'm looking for mo...","[84779, 203371, 165710, 122159, 151313, 191602]","B: Hi there, how are you? I'm looking for movi..."
1,1,2,"[(B: Hi, did you see scarface ?, [(196336, 1)]...","[204322, 114851, 204292, 128905, 145455, 19633...","B: Hi, did you see scarface ? /n A: Yes it was..."
2,2,3,[(B: avatar It is a nice movie. What say? avat...,"[155969, 84001, 204292, 90950, 147598, 163606,...",B: avatar It is a nice movie. What say? avatar...
3,3,4,[(B: I'm in the mood to watch a romantic comed...,[],B: I'm in the mood to watch a romantic comedy....
4,4,7,"[(A: Hello, None), (B: Hello What movie would ...","[184418, 119144, 204331, 125431, 136632, 141788]",A: Hello /n B: Hello What movie would you sugg...
...,...,...,...,...,...
10001,10001,936,"[(A: hi, None), (B: Hi !! have a good day whic...","[155632, 204292, 85036, 96852]",A: hi /n B: Hi !! have a good day which kind o...
10002,10002,935,"[(A: hello there!, None), (B: Hello!, None), (...","[101264, 183056, 193669, 204974]",A: hello there! /n B: Hello! /n A: can I help ...
10003,10003,952,"[(A: Hello!, None), (B: Hai, None), (A: How ca...","[192160, 184418, 182404, 195979, 136877, 20497...",A: Hello! /n B: Hai /n A: How can I help you? ...
10004,10004,953,"[(A: What type of movies do you like?, None), ...","[83552, 204899, 204974, 205934]",A: What type of movies do you like? /n B: Hi! ...


In [ ]:
def process_movie_db(movie_db):
    new_movie_db = {}
    for k in movie_db.keys():
        movie_db_lst = movie_db[k].split("[SEP]")
        meta_data = "Movie title: " + movie_db_lst[0].strip() + "\n" + "Actors: " + movie_db_lst[1].strip() + "\n" + "Directors: " + movie_db_lst[2].strip() + "\n" + "Genre: " + movie_db_lst[3].strip() + "\n" + "Plot: " + movie_db_lst[4].strip()
        new_movie_db[k] = meta_data
    return new_movie_db
movie_db = process_movie_db(movie_db)

In [11]:
train_embeddings = model.encode(train_df['message'], convert_to_tensor=True)
# test_embeddings = model.encode(test_df['message'])

movie_embeddings = model.encode(list(movie_db.values()), convert_to_tensor=True)

In [12]:
import torch.nn.functional as F

def get_longest_conversations(df, item_keys):
    results = {}
    
    for movie_id in item_keys:
        # Filter rows where `movie_id` is in `all_movie_id`
        filtered_df = df[df["all_movie_id"].apply(lambda x: movie_id in x)]
        
        if not filtered_df.empty:
            # Find the row with the longest conversation in `masked_messages`
            longest_row = filtered_df.loc[filtered_df["message"].str.len().idxmax()]
            results[movie_id] = longest_row["message"]
    
    return results


def get_sim_conversations(df, item_keys, model):
    results = {}
    
    for i, movie_id in enumerate(item_keys):
        # Filter rows where `movie_id` is in `all_movie_id`
        filtered_df = df[df["all_movie_id"].apply(lambda x: movie_id in x)]
        
        if not filtered_df.empty:
            conv_embeddings = train_embeddings[list(filtered_df['conv_id'])]
            item_embeddings = movie_embeddings[i].unsqueeze(0)
    
            conv_embeddings = F.normalize(conv_embeddings, p=2, dim=1)
            item_embeddings = F.normalize(item_embeddings, p=2, dim=1)
    
            similarities = torch.matmul(conv_embeddings, item_embeddings.T).squeeze()
    
            most_similar_index = torch.argmax(similarities, dim=0).item()
    
            results[movie_id] = df['message'][filtered_df.index[most_similar_index]]
    
    return results

result = get_sim_conversations(train_df, list(movie_db.keys()), model)
# result = get_longest_conversations(train_df, list(movie_db.keys()))


In [13]:
for id in result:
    movie_db[id] = "Example Conversation: " + result[id] + '/n ' + movie_db[id]
len(movie_db)

6924

In [14]:
movie_db[list(movie_db.keys())[0]]

"Movie title: headhunter\nActors: Lars Mikkelsen, Charlotte Munck, Burkhard Forstreuter\nDirectors: Rumle Hammerich\nGenre: Thriller\nPlot: Martin Vinge, a 35-year-old former journalist turned headhunter, is hired by N.F. Sieger, 85-year-old CEO of Denmark's largest shipping company and oil empire, to find an alternative heir to the firm, as his son Daniel is no longer the preferred choice. This decision sets off a complex power struggle, entangling Martin in a ruthless game that affects his personal life and relationships."

In [16]:
torch.save(movie_db, "/home/pc/Documents/CRS/data/full_redial/concat_movie_db")